# Quick MNIST

In [4]:
import pickle, gzip
from pathlib import Path
from mnist_sample1 import *
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

bs=64
lr=0.1
epochs=4
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = nn.Sequential(
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0),-1))
).to(dev)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

def preprocess(x,y): return x.view(-1,1,28,28).to(dev),y.to(dev)
def get_dataloader(x,y,bs,shuffle):
    ds = TensorDataset(*map(tensor, (x,y)))
    dl = DataLoader(ds, batch_size=bs, shuffle=False)
    dlwrap = WrappedDataLoader(dl, preprocess)
    return dlwrap

with gzip.open('data/mnist.pkl.gz', 'rb') as f:
    ((train_x, train_y), (valid_x, valid_y), _) = pickle.load(f, encoding='latin-1')

train_dl = get_dataloader(train_x, train_y, bs,   shuffle=False)
valid_dl = get_dataloader(valid_x, valid_y, bs*2, shuffle=True )

fit(epochs, model, F.cross_entropy, opt, train_dl, valid_dl)

0 0.43608038740158084
1 0.2648737972021103
2 0.23898396736979485
3 0.24525080618560313
